In [41]:
import numpy as np

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux

FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [42]:
    s_matrix = np.array([[-1, 1, 1, 0, 0, 0, 0, 0, 0],
                         [0, 0, 0, -1, 1, 0, 0, 0, 0],
                         [0, 0, -1, 0, -1, 1, 0, 0, 0],
                         [0, 0, 0, 0, 0, -1, -1, 1, 1]]).T

    metabolites = ["CYS[c]", "L-ALPHA-ALANINE[c]", "S-SULFANYL-IscS[c]", "FE+2[c]", "DISORDERED-FdS-IscU[c]", "S-SULFANYL-DISORDERED-FdS-IscU[c]", "ATP[c]", "ADP[c]", "CPD-6[c]"]
    reactions = ["cysteine-desulfurase-FeS-cluster-assembly", "iron-transport-FeS-cluster-assembly", "complex-formation-FeS-cluster-assembly", "cluster-transport-FeS-cluster-assembly"]
    homeostatic_metabolites = {"CPD-6[c]": 1} # Demand has to be added for everything in the pathway

    exchanges = {"CYS[c]", "FE+2[c]", "ATP[c]", "ADP[c]", "L-ALPHA-ALANINE[c]"}
    uptakes = {"CYS[c]", "FE+2[c]", "ATP[c]"}

    model = NetworkFlowModel(stoich_arr=s_matrix, reactions=reactions,
                             metabolites=metabolites, homeostatic_metabolites=list(homeostatic_metabolites.keys()),
                             kinetic_reactions=None)

    model.set_up_exchanges(exchanges=exchanges, uptakes=uptakes)

    solution: FlowResult = model.solve(homeostatic_targets=list(homeostatic_metabolites.values()),
                                       objective_weights={'secretion': 0.01, 'efficiency': 0.0001},
                                       # 0.01, 0.0001
                                       upper_flux_bound=100, solver=cp.GLOP)

    print(solution.velocities)
    assert np.isclose(solution.velocities, np.array([1, 1, 1, 1])).all() == True, "Network flow toy model did not converge to correct solution."

[1. 1. 1. 1.]
